## Resultados:

Se utilizó como corpus el cuento "The Shadow", tomando cada oración del mismo como documentos. A partir de ello, se utilizó la frase "The learned man from the cold lands", que forma parte del siguiente documento (oración):

"The learned man from the cold lands he was a young man, and seemed to be a clever man sat in a glowing oven; it took effect on him, he became quite meagre even his shadow shrunk in, for the sun had also an effect on it. It was first towards evening when the sun was down, that they began to freshen up again."

A partir de eso, se encontró que la cantidad de dosumentos fue de 212. Además, tomando el percentil 90 para la longitud de las sentencias, se llegó a que el tamaño máximo del contexto es de 53 y un vocabulario (tokenizado por palabras) de 1250 elementos.

Para generar predicciones sobre esto, se experimentó con 3 arquitecturas diferentes: RNN, GRU y LSTM. A su vez, se obtuvieron resultados para diferentes combinación de hiperparámetros ó técnicas. Entre ellos, Si la técnica de selección de resultado se realiza con "greed search" o "beam search" con diferentes valores de temperatura y modos (estocástico o determinísticos).

A continuación se presentan los resultados.

| Modelo | Nº de Beams | Nº de palabras | Temp. | Modo |           Sentencia                 | Predicción (con más probabilidad)               |
|--------|-------------|----------------|-------|------|-------------------------------------|-------------------------------------------------|
|  LSTM  |     10      |       6        |   1   | STO  | The learned man from the cold lands | and they had one in called                      |
|  LSTM  |     10      |       6        |   2   | STO  | The learned man from the cold lands | she not wise she be maiden                      |
|  LSTM  |     10      |       6        |   10  | STO  | The learned man from the cold lands | composed shop pounds bid coffee cracks          |
|  LSTM  |     10      |       6        |   100 | STO  | The learned man from the cold lands | much accepted health but extraordinary windows  |
|  GRU   |     10      |       6        |   1   | STO  | The learned man from the cold lands | lands and so he he the was                      |
|  GRU   |     10      |       6        |   2   | STO  | The learned man from the cold lands | from home ever him yes acquaintance             |
|  GRU   |     10      |       6        |   10  | STO  | The learned man from the cold lands | down above awake sound when him                 |
|  GRU   |     10      |       6        |   100 | STO  | The learned man from the cold lands | practised last prison beard certainly           |
|  RNN   |     10      |       6        |   1   | STO  | The learned man from the cold lands | you me in the sun talked                        |
|  RNN   |     10      |       6        |   2   | STO  | The learned man from the cold lands | with me as it was watering                      |
|  RNN   |     10      |       6        |   10  | STO  | The learned man from the cold lands | singular in perceptible hurrah certainly finest |
|  RNN   |     10      |       6        |   100 | STO  | The learned man from the cold lands | ascends know humor pronoun from burning         |

*Tabla 1: Sentencia más probable considerando diferentes arquitecturas y temperaturas.*

Por otro lado, se realizó un experimento que busca la predicción considerando la técnica "Beam Search" es su modo determinista:

| Modelo | Nº de Beams | Nº de palabras | Temp. | Modo |           Sentencia                 | Predicción (con más probabilidad)               |
|--------|-------------|----------------|-------|------|-------------------------------------|-------------------------------------------------|
|  LSTM  |     1       |       1        |   1   | DET  | The learned man from the cold lands | and                                             |
|  GRU   |     1       |       1        |   1   | DET  | The learned man from the cold lands | and                                             |
|  RNN   |     1       |       1        |   1   | DET  | The learned man from the cold lands | for                                             |

*Tabla 2: Sentencia de predicción considerando técnica beam search en su modo determinista, para diferentes arquitecturas.*

Por último, se realizó un experimento con la técnica "Greed Search" con diferentes arquetecturas:

| Modelo | Nº de palabras |           Sentencia                 | Predicción (con más probabilidad)               |
|--------|----------------|-------------------------------------|-------------------------------------------------|
|  LSTM  |       1        | The learned man from the cold lands | and                                             |
|  GRU   |       1        | The learned man from the cold lands | and                                             |
|  RNN   |       1        | The learned man from the cold lands | for                                             |

*Tabla 3: Sentencia de predicción considerando técnica greed search.*

Comparando las tres arquitecturas, los tamaños de los modelos fue el siguiente:

| modelo | cantidad de parametros | tamaño (MB) |
|--------|------------------------|-------------|
| LSTM   |       329701           |     1,26    |
| GRU    |       295101           |     1,13    |
| RNN    |       224101           |     0,86    |

 *Tabla 4: Tamaños y pesos entre los diferentes modelos.*

## Conclusiones:

#### Impacto de las Arquitecturas (RNN, LSTM y GRU):

La arquitectura LSTM maneja mejor las dependencias de largo plazo en secuencias, lo que puede observarse en las predicciones generadas en el modo estocástico (Tabla 1). Sin embargo, con temperaturas altas (como 10 y 100), las predicciones de LSTM se vuelven incoherentes, generando frases sin sentido como "composed shop pounds bid coffee cracks". Esto sugiere que, aunque LSTM es efectiva para dependencias largas, es sensible a la variabilidad introducida por altas temperaturas. Por otro lado, la arquitectura GRU presenta un rendimiento similar al de LSTM en términos de capacidad de generación, pero tiende a tener una salida menos coherente en ciertas configuraciones, como en los ejemplos de temperatura elevada ("practised last prison beard certainly"). La GRU, siendo más simple que LSTM, conserva un balance entre memoria y procesamiento, pero puede ser menos efectiva para dependencias largas. En cuanto a la RNN básica, produce frases menos coherentes, especialmente a temperaturas más altas, como "ascends know humor pronoun from burning". Esto confirma la limitación de las RNN en capturar dependencias largas debido al problema del desvanecimiento del gradiente, lo que reduce su capacidad para producir secuencias contextualmente relevantes en textos extensos.

#### Influencia de la Técnica de Generación (Beam Search vs. Greedy Search):

Como se observa en la Tabla 2, con solo un beam (en su búsqueda determinista), tanto LSTM como GRU predicen "and" como la palabra más probable después de la frase inicial. Esto sugiere que, en configuraciones deterministas y con pocos beams, los modelos tienden a elegir las palabras más probables, aunque no siempre logren construir frases largas y coherentes. Al aumentar el número de beams, es posible que se logre una mayor diversidad en la generación, pero se requiere más recursos computacionales. La técnica Greedy Search en la Tabla 3 también da resultados muy similares a Beam Search determinista con un solo beam, indicando que ambos métodos tienden a capturar la palabra de mayor probabilidad en cada paso. Sin embargo, por concepto Greedy Search es menos flexible y limita la exploración de palabras alternativas, lo que puede restringir la creatividad y coherencia en textos largos.

#### Influencia de la Temperatura en las Predicciones Estocásticas:

En las configuraciones estocásticas de Beam Search (Tabla 1), se observa que conforme aumenta la temperatura, el texto generado pierde coherencia. A temperaturas altas (por ejemplo, 10 y 100), el modelo genera secuencias aleatorias e incoherentes en todos los modelos. Esto ocurre porque una temperatura alta reduce el peso de las palabras más probables, permitiendo que palabras menos probables tengan más impacto, lo que genera mayor diversidad pero disminuye la coherencia.

#### Comparación de Tamaños de los Modelos:

Los tamaños de los modelos en términos de parámetros y MB indican que LSTM es el modelo más grande, seguido de GRU y luego RNN. Esto es consistente con la complejidad de cada arquitectura Las LSTM tienen más parámetros debido a sus múltiples "puertas", mientras que las GRU, al ser más simples, requieren menos memoria y procesamiento. En entornos con restricciones de memoria, GRU podría ser una alternativa efectiva que ofrezca una buena relación entre rendimiento y costo computacional, aunque para resultados de alta calidad en contextos largos, LSTM podría ser preferible.